In [ ]:
# WARNING: advised to install a specific version, e.g. qrules==0.1.2
%pip install -q qrules[doc,viz] IPython

In [ ]:
import os

STATIC_WEB_PAGE = {"EXECUTE_NB", "READTHEDOCS"}.intersection(os.environ)

```{autolink-concat}
```

# LS-couplings

The {func}`.spin_conservation` rule is one of the more complicated checks in the {mod}`.conservation_rules` module. It provides an implementation of $LS$-couplings, which is a procedure to determine which values for **total angular momentum $L$** and **coupled spin $S$** are allowed in an interaction node. In this notebook, we illustrate this procedure with the following decay chain as an example:

$$
J/\psi \to \Sigma^+ \bar\Sigma(1670)^-, \quad \bar\Sigma(1670)^- \rightarrow \bar p K^0.
$$

In this decay chain, there are two decay nodes that we investigate separately. In addition, both decays are mediated interactions by the strong force, which means there is also **parity conservation**.

In the following derivations, the {attr}`.Particle.spin` and {attr}`.Particle.parity` values are of importance:

In [ ]:
from IPython.display import Math

import qrules

PDG = qrules.load_pdg()
particle_names = [
    "J/psi(1S)",
    "Sigma+",
    "Sigma(1670)~-",
    "p~",
    "K0",
]
latex_expressions = []
for name in particle_names:
    particle = PDG[name]
    parity = "+" if particle.parity > 0 else "-"
    if particle.spin.is_integer():
        spin = int(particle.spin)
    else:
        nominator = int(particle.spin * 2)
        spin = Rf"\tfrac{{{nominator}}}{2}"
    latex_expressions.append(f"{particle.latex}={spin}^{parity}")
Math(R"\qquad ".join(latex_expressions))

## Procedure

Imagine we have a two-body decay of $p_0\rightarrow p_1p_2$. We denote the {attr}`.Spin.magnitude` of each particle $p_i$ as $s_i$ and their {attr}`~.Particle.parity` as $\eta_i$. The values for $L$ and $S$ can now be determined as follows:

1. Determine all values for $S$ that satisfy $\left| s_1-s_2 \right| \le S \le s_1+s_2$. The difference between each value for $S$ has to integer, so $S = \left| s_1-s_2 \right|, \left| s_1-s_2 \right|+1, \dots, s_1+s_2$.
2. Determine all values for $L$ that satisfy $\left| L-S \right| \le s_0 \le L+S$, with $L$ being a non-negative integer.
3. If there is parity conservation, $L$ has to satisfy an additional constraint: $\eta_0 = \eta_1\cdot\eta_2\cdot(-1)^L$.

## $J/\psi \to \Sigma^+\bar\Sigma(1670)^-$

The spin and parity of each particle in the first transition can be summarized as $1^-\to\frac{1}{2}^+\frac{3}{2}^+$. Following step 1 in the {ref}`procedure <usage/ls-coupling:procedure>`, we get:

$$
\begin{eqnarray}
\left|s_{\Sigma^+} - s_{\bar\Sigma(1670)^-}\right| & \le S & \le s_{\Sigma^+} + s_{\bar\Sigma(1670)^-} \\
\left|\tfrac{1}{2}-\tfrac{3}{2}\right| & \le S & \le \tfrac{1}{2} + \tfrac{3}{2} \\
1 & \le S & \le 2
\end{eqnarray}
$$

$$
\Rightarrow S=1 \quad \text{or} \quad S=2
$$

Next, we determine the allowed total angular momentum values $L$ with {ref}`step 2 <usage/ls-coupling:procedure>`:

$$
\begin{eqnarray}
|L-S| & \le s_{J/\psi} & \le L+S \\
|L-S| & \le 1 & \le L+S
\end{eqnarray}
$$

$$
\Rightarrow \begin{cases}
L=0,1,2 & \text{if} & S=1\\
L=1,2,3 & \text{if} & S=2.
\end{cases}
$$

So in total, we have 6 $LS$-combinations:

$$
(L,S) = (0,1), (1,1), (2,1), (1,2), (2,2), (3,2).
$$

This decay however goes via the strong force. This means that parity has to be conserved and we have to follow {ref}`step 3 <usage/ls-coupling:procedure>`:

$$
\begin{eqnarray}
\eta_{J/\psi} & = & \eta_{\Sigma^+} \cdot \eta_{\bar\Sigma(1670)^-} \cdot(-1)^L \\
(-1) & = & (+1)\cdot(+1)\cdot(-1)^L \\
(-1) & = & (-1)^{L}.
\end{eqnarray}
$$

From this, we can easily see that only odd $L$ values are possible, which leaves us with **3 $LS$-combinations**:

$$
(L,S) = (1,1), (1,2), (3,2).
$$

## $\bar \Sigma(1670)^-\to \bar pK^0$

The second part of the decay chain can be expressed as $\frac{3}{2}^+ \to \frac{1}{2}^- 0^-$. Following {ref}`step 1 <usage/ls-coupling:procedure>`, we see:

$$
\begin{eqnarray}
|s_{\bar p} - s_{K^0}| & \le S  & \le s_{\bar p} + s_{K^0} \\
\left|\tfrac{1}{2}-0 \right| & \le S & \le \tfrac{1}{2} + 0
\end{eqnarray}
$$

$$
\Rightarrow S = \tfrac{1}{2}.
$$

This time, only one coupled spin value is allowed. That allows for the following values of $L$:

$$
\begin{eqnarray}
|L-S| & \le s_0 & \le L+S \\
\left|L-\tfrac{1}{2}\right| & \le \tfrac{3}{2} & \le L+\tfrac{1}{2}.
\end{eqnarray}
$$

$$
\Rightarrow L = 1,2
$$

By now, only two $LS$-combinations are possible:

$$
(L,S)=\left(1,\tfrac{1}{2}\right), \left(2,\tfrac{1}{2}\right).
$$

This again is a strong interaction, which means we have to check for parity conservation.

$$
\begin{eqnarray}
    \eta_{\bar \Sigma(1670)^-} & = & \eta_{\bar p}\cdot\eta_{K^0}\cdot(-1)^L\\
    (+1) & = & (-1)\cdot(-1)\cdot(-1)^L\\
    (+1) & = & (-1)^L.
\end{eqnarray}
$$

Again, it is clear that only even $L$'s are allowed. This means that **only one $LS$-combination** is possible:

$$
(L,S)=\left(2,\tfrac{1}{2}\right)
$$

## Check with QRules

Finally, let's use {func}`.generate_transitions` to check whether the allowed $LS$-couplings are found by {mod}`qrules` as well. Note that we have to increase the maximum angular momentum to find the $(L,S)=(3,2)$ combination as well.

In [ ]:
import logging

import graphviz

LOGGER = logging.getLogger()
LOGGER.setLevel(logging.ERROR)

reaction = qrules.generate_transitions(
    initial_state="J/psi(1S)",
    final_state=["K0", "Sigma+", "p~"],
    allowed_intermediate_particles=["Sigma(1670)"],
    allowed_interaction_types="strong",
    max_angular_momentum=3,
)
dot = qrules.io.asdot(reaction, render_node=True, strip_spin=True)
graphviz.Source(dot)